In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
.filter(f"file_date = '{v_file_date}'") 

In [0]:
race_year_list = df_column_to_list(race_results_df,"race_year")

In [0]:
from pyspark.sql.functions import col

race_results = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
.filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.functions import sum, when, col, count

driver_standing_df = race_results \
.groupBy("race_year","driver_name","driver_nationality") \
.agg(sum("points").alias("total_points"),
     count(when(col("position") == 1,True)).alias("wins"))


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

driver_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))

In [0]:
final_df = driver_standing_df.withColumn("rank", rank().over(driver_rank_spec))

In [0]:
merge_condition = "target.race_year = source.race_year and target.driver_name = source.driver_name"
merge_delta_data(final_df, "f1_presentation", "driver_standings", presentation_folder_path, merge_condition, "race_year")

In [0]:
%sql
/*select race_year, sum(total_points) as total_points, count(1)
 from f1_presentation.driver_standings
group by race_year
order by race_year desc, total_points desc*/

race_year,total_points,count(1)
2021,204.0,20
2020,1734.0,23
2019,2140.0,20
2018,2121.0,20
2017,2020.0,25
2016,2121.0,24
2015,1919.0,22
2014,2020.0,24
2013,1919.0,23
2012,2020.0,25
